# **ETL Case Study**

### Library Importation

In [1]:
import numpy as np
import pandas as pd
import pymysql


import warnings
warnings.filterwarnings("ignore")


pd.set_option('display.max_columns', None)

## Extract

In [2]:
data = pd.read_excel('Depressed!!!.xlsx')

In [3]:
data.head(3)

,Name,Gender,Age,City,Status,Profession,Mental Pressure,Work/Academic Pressure,Performance Rating,Mental Satisfaction,Performance Satisfaction,Sleep Duration,Dietary Habit,Degree,Blood Group,Mood Swings,Working Hours,Financial Stress,Family History of Depression,Smoking,Overthinking,Depression,Date Of Test
0,Person_1,LGBTQ,31,NaN,Working,Artist,4,3,10,5,1,7-8 hours,Unhealthy,Post Doc,O-,High,5,Yes,Yes,No,Normal,Yes,21
1,Person_2,Male,41,NaN,Working,Content Writer,4,1,7,5,2,7-8 hours,Moderate,Masters,AB-,Severe,2,No,No,Yes,Rarely,Yes,21
2,Person_3,LGBTQ,74,NaN,Working,Doctor,2,4,4,1,5,5-6 hours,Unhealthy,Masters,B+,Severe,15,Yes,Yes,No,Normal,Yes,19


## Transform

In [4]:
data.isnull().sum().sum()

13510

In [5]:
data = data.astype(object).where(pd.notnull(data),None)

In [6]:
data

,Name,Gender,Age,City,Status,Profession,Mental Pressure,Work/Academic Pressure,Performance Rating,Mental Satisfaction,Performance Satisfaction,Sleep Duration,Dietary Habit,Degree,Blood Group,Mood Swings,Working Hours,Financial Stress,Family History of Depression,Smoking,Overthinking,Depression,Date Of Test
0,Person_1,LGBTQ,31,None,Working,Artist,4,3,10,5,1,7-8 hours,Unhealthy,Post Doc,O-,High,5,Yes,Yes,No,Normal,Yes,21
1,Person_2,Male,41,None,Working,Content Writer,4,1,7,5,2,7-8 hours,Moderate,Masters,AB-,Severe,2,No,No,Yes,Rarely,Yes,21
2,Person_3,LGBTQ,74,None,Working,Doctor,2,4,4,1,5,5-6 hours,Unhealthy,Masters,B+,Severe,15,Yes,Yes,No,Normal,Yes,19
3,Person_4,LGBTQ,67,None,Student,Content Writer,2,3,7,5,4,More than 8 hours,Unhealthy,Masters,O-,Rare,2,Yes,No,Yes,Rarely,Yes,19
4,Person_5,Male,74,None,Student,Designer,2,3,6,5,3,5-6 hours,Moderate,Bachelor,O-,Normal,12,No,Yes,Yes,High,Yes,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,Person_14996,LGBTQ,35,None,Working,Engineer,5,5,4,3,4,More than 8 hours,Unhealthy,Schooling,B-,Rare,10,Yes,No,Yes,Rarely,Yes,20
14996,Person_14997,Male,67,None,Retired,Engineer,1,2,1,2,4,More than 8 hours,Unhealthy,Doctorate,B-,Rare,10,Yes,Yes,Yes,Rarely,Yes,12
14997,Person_14998,Female,75,None,Retired,Designer,5,4,4,1,2,More than 8 hours,Moderate,Bachelor,O-,Rare,12,No,No,No,Severe,Yes,29
14998,Person_14999,Male,74,New York,Student,Doctor,2,5,5,2,5,7-8 hours,Unhealthy,Masters,AB-,High,11,Yes,Yes,Yes,Severe,Yes,15


In [7]:
data.shape

(15000, 23)

In [8]:
pipeline = pymysql.connect(
    host = 'localhost',
    user= 'root',
    password= '14072003'

)

etl = pipeline.cursor()

## Load

In [9]:
# Quries for Database Creation and make it in Use

etl.execute('Drop Database if exists depression;')   # this is also helpful if incase we want any corrections or run all cell in ipynb file


etl.execute('Create Database depression;')
print('New Database named Depression Created;')


etl.execute('use depression;')
print('Depression Database is in use now;')

New Database named Depression Created;
Depression Database is in use now;


In [10]:
# Queries for Table Creation & data Insertion in it

etl.execute('''Create Table Dep
            (Name varchar(30),
            Gender varchar(30),
            Age int,
            City varchar(30),
            Status varchar(30),
            Profession text,
            Mental_Pressure int,
            Work_Academic_Pressure int,
            Performance_Rating int,
            Mental_Satisfaction int,
            Performance_Satisfaction int,
            Sleep_Duration varchar(30),
            Dietary_habit varchar(30),
            Degree text,
            Blood_Group varchar(30),
            Mood_Swings Text,
            Working_Hours int,
            Financial_Stress text,
            Family_History_of_Depression char(20),
            Smoking char(30),
            Overthinking char(30),
            Depression char(10),
            Date_of_Test int 
            );''')
print('Table Created Successfully')

insert_query = ("""insert into 
                Dep(Name, Gender, Age, City, Status, Profession, Mental_Pressure, Work_Academic_Pressure,
                Performance_Rating, Mental_Satisfaction, Performance_Satisfaction, Sleep_Duration, Dietary_habit,
                Degree, Blood_Group, Mood_Swings, Working_Hours, Financial_Stress, Family_History_of_Depression,
                Smoking, Overthinking, Depression, Date_of_Test)
                values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""")

for i in data.itertuples(index = False, name= None):
    etl.execute(insert_query, i)

print('Data is Succesfully Inserted into the tables')

Table Created Successfully
Data is Succesfully Inserted into the tables


In [11]:
# Let's Close the COnnections

pipeline.commit()
etl.close()
pipeline.close()

### Let's Solve Some Questions Related to the data fetching using SQL Query but on Python

In [12]:
# Lets reopen and set up the connection again

pipeline = pymysql.connect(host='localhost',
                           user = 'root',
                           password= '14072003')

etl = pipeline.cursor()

- 1. Count how many users have reported depression.

In [13]:
ans = etl.execute('Use depression;')

In [14]:
etl.execute("Select * from dep where depression = 'yes'")

14933

In [15]:
ans = pd.read_sql("Select * from dep where depression = 'yes';", pipeline)
ans

,Name,Gender,Age,City,Status,Profession,Mental_Pressure,Work_Academic_Pressure,Performance_Rating,Mental_Satisfaction,Performance_Satisfaction,Sleep_Duration,Dietary_habit,Degree,Blood_Group,Mood_Swings,Working_Hours,Financial_Stress,Family_History_of_Depression,Smoking,Overthinking,Depression,Date_of_Test
0,Person_1,LGBTQ,31,None,Working,Artist,4,3,10,5,1,7-8 hours,Unhealthy,Post Doc,O-,High,5,Yes,Yes,No,Normal,Yes,21
1,Person_2,Male,41,None,Working,Content Writer,4,1,7,5,2,7-8 hours,Moderate,Masters,AB-,Severe,2,No,No,Yes,Rarely,Yes,21
2,Person_3,LGBTQ,74,None,Working,Doctor,2,4,4,1,5,5-6 hours,Unhealthy,Masters,B+,Severe,15,Yes,Yes,No,Normal,Yes,19
3,Person_4,LGBTQ,67,None,Student,Content Writer,2,3,7,5,4,More than 8 hours,Unhealthy,Masters,O-,Rare,2,Yes,No,Yes,Rarely,Yes,19
4,Person_5,Male,74,None,Student,Designer,2,3,6,5,3,5-6 hours,Moderate,Bachelor,O-,Normal,12,No,Yes,Yes,High,Yes,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14928,Person_14996,LGBTQ,35,None,Working,Engineer,5,5,4,3,4,More than 8 hours,Unhealthy,Schooling,B-,Rare,10,Yes,No,Yes,Rarely,Yes,20
14929,Person_14997,Male,67,None,Retired,Engineer,1,2,1,2,4,More than 8 hours,Unhealthy,Doctorate,B-,Rare,10,Yes,Yes,Yes,Rarely,Yes,12
14930,Person_14998,Female,75,None,Retired,Designer,5,4,4,1,2,More than 8 hours,Moderate,Bachelor,O-,Rare,12,No,No,No,Severe,Yes,29
14931,Person_14999,Male,74,New York,Student,Doctor,2,5,5,2,5,7-8 hours,Unhealthy,Masters,AB-,High,11,Yes,Yes,Yes,Severe,Yes,15


- 2. Find average age of people experiencing depression.

In [16]:
pd.read_sql('Select avg(age) from dep where depression = "yes";', pipeline)

,avg(age)
0,48.9316


- 3. List distinct cities with more than 5 depressed individuals.

In [17]:
pd.read_sql('Select distinct city, count(*) from dep  where depression = "yes" group by 1 having count(*) > 5;', pipeline)

,city,count(*)
0,None,13449
1,Tokyo,355
2,New York,372
3,London,373
4,Bangalore,384


- 4. Top 3 professions with highest average mental pressure

In [18]:
pd.read_sql('Select profession, avg(mental_pressure) from dep group by 1 order by 2 desc limit 3;', pipeline)

,profession,avg(mental_pressure)
0,Pilot,3.0458
1,Artist,3.0335
2,Engineer,3.0211


- 5. Get number of people with ‘High’ work/academic pressure by gender.

In [19]:
pd.read_sql('select gender , count(*) from dep where work_academic_pressure = 5 group by 1; ', pipeline)

,gender,count(*)
0,LGBTQ,987
1,Female,972
2,Male,981


- 6. Find users whose mental satisfaction is below the overall average.

In [20]:
pd.read_sql("Select Name, mental_satisfaction from dep where mental_satisfaction < (select avg(mental_satisfaction) from dep) ", pipeline)

,Name,mental_satisfaction
0,Person_3,1
1,Person_7,1
2,Person_11,1
3,Person_12,2
4,Person_13,1
...,...,...
6064,Person_14991,1
6065,Person_14997,2
6066,Person_14998,1
6067,Person_14999,2


- 7. List individuals older than the average age of all ‘Students’.

In [21]:
pd.read_sql('Select Name, age from dep where age > (select avg(age) from dep where status = "Student")', pipeline)

,Name,age
0,Person_3,74
1,Person_4,67
2,Person_5,74
3,Person_6,54
4,Person_7,78
...,...,...
7357,Person_14991,52
7358,Person_14995,78
7359,Person_14997,67
7360,Person_14998,75


- 8. Get cities where max working hours > 8 using a subquery.

In [22]:
pd.read_sql("Select distinct city from dep where working_hours = 'More than 8 hours';", pipeline)

,city
0,None
1,London
2,Tokyo
3,New York
4,Bangalore


- 9. Find names of users whose performance rating is above their city’s average.

In [24]:
pd.read_sql("Select name, city , performance_rating from dep d\
             where performance_rating > (select avg(performance_rating) from dep as sub where sub.city = d.city);", pipeline)

,name,city,performance_rating
0,Person_25,Tokyo,10
1,Person_113,New York,10
2,Person_125,London,8
3,Person_155,Bangalore,6
4,Person_248,New York,6
...,...,...,...
732,Person_14947,New York,10
733,Person_14950,Bangalore,8
734,Person_14983,Tokyo,9
735,Person_14992,London,8


- 10. List users who sleep less than the average sleep duration for their gender.

In [25]:
pd.read_sql("Select name, sleep_duration, gender from dep d \
            where sleep_duration < (select avg(sleep_duration) from dep as sub where sub.gender = d.gender );", pipeline)

,name,sleep_duration,gender
0,Person_4,More than 8 hours,LGBTQ
1,Person_8,More than 8 hours,Female
2,Person_10,More than 8 hours,LGBTQ
3,Person_12,Less than 5 hours,LGBTQ
4,Person_15,Less than 5 hours,Female
...,...,...,...
7426,Person_14994,More than 8 hours,Male
7427,Person_14995,More than 8 hours,Female
7428,Person_14996,More than 8 hours,LGBTQ
7429,Person_14997,More than 8 hours,Male


- 11. Assign a rank to each person based on performance rating within their city

In [27]:
pd.read_sql(' select name,performance_rating, city, rank() over(partition by city order by performance_rating ) from dep ;', pipeline)

,name,performance_rating,city,rank() over(partition by city order by performance_rating )
0,Person_12056,1,None,1
1,Person_12585,1,None,1
2,Person_12173,1,None,1
3,Person_13553,1,None,1
4,Person_13974,1,None,1
...,...,...,...,...
14995,Person_13445,10,Tokyo,320
14996,Person_12405,10,Tokyo,320
14997,Person_12594,10,Tokyo,320
14998,Person_11141,10,Tokyo,320


- 12. Find the top performer in each profession using ROW_NUMBER().

In [28]:
pd.read_sql("Select name, performance_rating, row_number() over(partition by profession order by performance_rating) from dep", pipeline)

,name,performance_rating,row_number() over(partition by profession order by performance_rating)
0,Person_14825,1,1
1,Person_14643,1,2
2,Person_14874,1,3
3,Person_14961,1,4
4,Person_14649,1,5
...,...,...,...
14995,Person_9972,10,2225
14996,Person_10299,10,2226
14997,Person_9424,10,2227
14998,Person_9884,10,2228


- 13. Calculate the cumulative sum of performance satisfaction across all rows.

In [31]:
pd.read_sql("""SELECT 
    Name, 
    Performance_Satisfaction,
    SUM(Performance_Satisfaction) OVER (ORDER BY Name) AS Cumulative_Satisfaction
FROM dep;""", pipeline)

,Name,Performance_Satisfaction,Cumulative_Satisfaction
0,Person_1,1,1.0
1,Person_10,2,3.0
2,Person_100,1,4.0
3,Person_1000,1,5.0
4,Person_10000,4,9.0
...,...,...,...
14995,Person_9995,3,45364.0
14996,Person_9996,1,45365.0
14997,Person_9997,2,45367.0
14998,Person_9998,5,45372.0


- 14. Get the average mental pressure per city and show it alongside each person (using OVER PARTITION BY).

In [34]:
pd.read_sql("""SELECT 
    Name, 
    Mental_Pressure, city,
    AVG(Mental_Pressure) OVER (partition by city ORDER BY Name) AS Avg_Satisfaction
FROM dep;""", pipeline)

,Name,Mental_Pressure,city,Avg_Satisfaction
0,Person_1,4,None,4.0000
1,Person_10,5,None,4.5000
2,Person_100,2,None,3.6667
3,Person_1000,5,None,4.0000
4,Person_10000,2,None,3.6000
...,...,...,...,...
14995,Person_9819,3,Tokyo,2.9688
14996,Person_9857,5,Tokyo,2.9746
14997,Person_9874,5,Tokyo,2.9803
14998,Person_9908,4,Tokyo,2.9831


- 15. List users whose overthinking is marked 'severe', with their rank in working hours by city.

In [35]:
pd.read_sql("Select name, city, rank() over(partition by city order by working_hours) from dep where overthinking = 'severe'; ", pipeline)

,name,city,rank() over(partition by city order by working_hours)
0,Person_10369,None,1
1,Person_12,None,1
2,Person_7445,None,1
3,Person_12313,None,1
4,Person_8453,None,1
...,...,...,...
3772,Person_11939,Tokyo,96
3773,Person_10658,Tokyo,96
3774,Person_7586,Tokyo,96
3775,Person_2062,Tokyo,96
